In [1]:
# step by step guide: https://towardsdatascience.com/multiclass-classification-algorithm-from-scratch-with-a-project-in-python-step-by-step-guide-485a83c79992

In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
#from sklearn.experimental import enable_halving_search_cv
#from sklearn.model_selection import HalvingRandomSearchCV


In [3]:
#test_run = True
test_run = False

In [4]:
df = pd.read_csv("Temp/df_proc_train_set.csv")

In [5]:
df['sol_category'] = pd.to_numeric(df['sol_category'], downcast='integer')
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [6]:
if test_run:
    df0 = pd.DataFrame()
    #index = 0
    for index in tqdm(df.index):
        if index % 100 == 0:
            #row_i = df.loc[index, :]
            df0 = pd.concat([df0, df.loc[index, :]], axis=1) #, row_i])
    df0 = df0.T
    df0['sol_category'].value_counts()
    df = df0

In [7]:
df

,Id,smiles,sol_category,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,EOS12286,Cc1nc(N2CCN(C(=O)Nc3ccc(F)cc3F)CC2)cc(-n2ccnc2)n1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,EOS85869,CCN(CC)[C@H]1CCN(C(=O)Cc2nc(C(C)C)c(C)s2)C1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,EOS85435,CNC(=O)CNC(=O)c1c(-n2cccc2)sc(C)c1C,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,EOS102302,CC(C)(C)c1ccc(CSc2cnn(C(C)(C)C)c(=O)c2Cl)cc1,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,EOS64213,CC[C@H](NC(=O)c1ccnc(-n2ccnc2)c1)c1ccccc1OC,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,EOS37839,O=C(NCCCc1nc(=O)[nH][nH]1)[C@H]1CCC(F)(F)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,EOS2088,Cc1ccc(C(=O)NC2CCCC2)cc1S(=O)(=O)N1CCOCC1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,EOS10587,COCCN1CCC(CN(C)S(=O)(=O)c2cccc(C(F)(F)F)c2)C1,2,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,EOS40533,O=C(Nc1ccc(F)cc1)NC1CCN(C(=O)Cc2cnn(-c3ccccc3)...,2,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [8]:
#y = pd.read_excel(xl, 'y', header = None)

In [9]:
# hypothesis fuction
# takes input variables and theta, return calculated output variable

In [10]:
print(len(df))
#print(len(y))
X = df.copy(deep=True)
X.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X

70711


,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Only exfp

In [11]:
X_ecfp = df.copy(deep=True)
drop_list_desc = [column for column in X_ecfp.columns if (column.startswith('example_') or column.startswith('rdkit_desc_'))]

# Drop features 
X_ecfp.drop(drop_list_desc, axis=1, inplace=True)
X_ecfp.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_ecfp

,ecfp_0,ecfp_1,ecfp_2,ecfp_3,ecfp_4,ecfp_5,ecfp_6,ecfp_7,ecfp_8,ecfp_9,...,ecfp_4086,ecfp_4087,ecfp_4088,ecfp_4089,ecfp_4090,ecfp_4091,ecfp_4092,ecfp_4093,ecfp_4094,ecfp_4095
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70707,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70708,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
70709,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Only descriptors

In [12]:
X_desc = df.copy(deep=True)
drop_list_MFP = [column for column in X_desc.columns if column.startswith('ecfp_')]

# Drop features 
X_desc.drop(drop_list_MFP, axis=1, inplace=True)
X_desc.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_desc

,example_SlogP,example_SMR,example_TPSA,example_NumLipinskiHBA,example_NumLipinskiHBD,example_NumRotatableBonds,example_NumHBA,example_NumAmideBonds,example_NumHeteroAtoms,example_NumAtoms,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,2.60302,103.0537,79.18,8,1,3,6,2,10,48,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,3.06012,92.5500,36.44,4,0,6,4,1,5,51,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1.63144,79.5199,63.13,5,2,4,4,2,6,37,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,5.24150,103.2060,34.89,3,0,3,4,0,5,49,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.15700,94.8512,69.04,6,1,6,5,1,6,45,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0.58220,62.8271,90.64,6,3,5,3,1,8,35,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,1.68832,90.4790,75.71,6,1,4,4,1,7,48,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,2.29420,87.6828,49.85,5,0,7,4,0,9,48,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,3.36660,115.4244,79.26,7,2,5,4,3,8,55,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### only example descriptors

In [13]:
X_example = X_desc.copy(deep=True)
drop_list_ex = [column for column in X_example.columns if column.startswith('rdkit_desc_')]

# Drop features 
X_example.drop(drop_list_ex, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_example

,example_SlogP,example_SMR,example_TPSA,example_NumLipinskiHBA,example_NumLipinskiHBD,example_NumRotatableBonds,example_NumHBA,example_NumAmideBonds,example_NumHeteroAtoms,example_NumAtoms,...,example_NumSaturatedRings,example_NumAliphaticRings,example_NumAromaticHeterocycles,example_NumSaturatedHeterocycles,example_NumAliphaticHeterocycles,example_NumAromaticCarbocycles,example_NumSaturatedCarbocycles,example_NumAliphaticCarbocycles,example_FractionCSP3,example_HallKierAlpha
0,2.60302,103.0537,79.18,8,1,3,6,2,10,48,...,1,1,2,1,1,1,0,0,0.263158,-3.56
1,3.06012,92.5500,36.44,4,0,6,4,1,5,51,...,1,1,1,1,1,0,0,0,0.764706,-0.94
2,1.63144,79.5199,63.13,5,2,4,4,2,6,37,...,0,0,2,0,0,0,0,0,0.285714,-2.08
3,5.24150,103.2060,34.89,3,0,3,4,0,5,49,...,0,0,1,0,0,1,0,0,0.473684,-1.26
4,3.15700,94.8512,69.04,6,1,6,5,1,6,45,...,0,0,2,0,0,1,0,0,0.210526,-3.15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,0.58220,62.8271,90.64,6,3,5,3,1,8,35,...,1,1,1,0,0,0,1,1,0.727273,-1.73
70707,1.68832,90.4790,75.71,6,1,4,4,1,7,48,...,2,2,0,1,1,1,1,1,0.588235,-1.44
70708,2.29420,87.6828,49.85,5,0,7,4,0,9,48,...,1,1,0,1,1,1,0,0,0.625000,-1.16
70709,3.36660,115.4244,79.26,7,2,5,4,3,8,55,...,1,1,1,1,1,2,0,0,0.260870,-3.68


#### only rdkit descriptors

In [14]:
X_rdd = X_desc.copy(deep=True)
drop_list_rd = [column for column in X_rdd.columns if column.startswith('example_')]

# Drop features 
X_rdd.drop(drop_list_rd, axis=1, inplace=True)
#X_example.drop(columns=['Id', 'smiles', 'sol_category'], inplace=True)
X_rdd

,rdkit_desc_MaxEStateIndex,rdkit_desc_MinEStateIndex,rdkit_desc_MinAbsEStateIndex,rdkit_desc_qed,rdkit_desc_MaxPartialCharge,rdkit_desc_MinPartialCharge,rdkit_desc_MaxAbsPartialCharge,rdkit_desc_FpDensityMorgan1,rdkit_desc_FpDensityMorgan2,rdkit_desc_FpDensityMorgan3,...,rdkit_desc_fr_quatN,rdkit_desc_fr_sulfide,rdkit_desc_fr_sulfonamd,rdkit_desc_fr_sulfone,rdkit_desc_fr_term_acetylene,rdkit_desc_fr_tetrazole,rdkit_desc_fr_thiazole,rdkit_desc_fr_thiophene,rdkit_desc_fr_unbrch_alkane,rdkit_desc_fr_urea
0,13.751480,-0.805113,0.046085,0.732517,0.321549,-0.352889,0.352889,1.068966,1.827586,2.517241,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,12.524029,0.234919,0.234919,0.807124,0.229004,-0.340705,0.340705,1.409091,2.090909,2.636364,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,12.317465,-0.223009,0.019061,0.899102,0.254519,-0.357642,0.357642,1.250000,1.900000,2.500000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,12.357761,-0.382235,0.149177,0.698158,0.286663,-0.266023,0.286663,1.083333,1.666667,2.166667,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,12.704152,-0.151369,0.131009,0.750601,0.251455,-0.496395,0.496395,1.280000,2.120000,2.840000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70706,12.941416,-2.697880,0.208737,0.685975,0.360787,-0.355849,0.360787,1.473684,2.263158,2.842105,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
70707,12.866088,-3.602640,0.194141,0.894636,0.251038,-0.378711,0.378711,1.208333,1.875000,2.541667,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70708,12.805377,-4.572020,0.155028,0.728420,0.415953,-0.383360,0.415953,1.360000,2.080000,2.720000,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
70709,12.950221,-0.351472,0.008955,0.663055,0.318953,-0.342256,0.342256,1.032258,1.741935,2.451613,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [15]:
y = df.iloc[:, 2]

In [16]:
#y

In [17]:
y.unique()

array([0, 1, 2], dtype=int8)

## StandardScaler

In [18]:
#scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(X)
#X = scaler.fit_transform(X)

## split

In [19]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, train_size=0.8, random_state=1,  shuffle=True, stratify=y)

In [20]:
# find out where NaN is, remove in processing

## multi-class logistic regression

In [21]:
#clf = LogisticRegression(random_state=0, max_iter=1000, verbose=1).fit(X_train, y_train)

In [22]:
#clf.predict(X[:2, :])

In [23]:
#clf.predict_proba(X[:2, :])

In [24]:
#y_prediction = clf.predict(X_test)

In [25]:
#df_y = pd.DataFrame(y_prediction)

In [26]:
# Stricter feature selection
### 1. just use ecfp/rdkit+example
### 2. Hyperparatuning (StratifiedKfold+RandomizedSearchCV on X_train and y_train) (max_iter, solver, penalty,C)

# for dataset over all_dataset
#   hyperparameter tuning()

In [27]:
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.metrics import cohen_kappa_score
from numpy import linspace

In [28]:
model = LogisticRegression(random_state=0)

parameters = {
    'max_iter' : [2000], #range(1000, 2000),
    #'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    #'penalty' : ['l2', 'none'],
    'C' : linspace(0.1, 1.0, num=50)
    }

In [29]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

import warnings
warnings.filterwarnings("ignore", category=UserWarning)


In [30]:
all_dataset = list()
all_dataset.append(X)
all_dataset.append(X_ecfp)
all_dataset.append(X_desc)
#all_dataset.append(X_example)
#all_dataset.append(X_rdd)

qwck_scorer = make_scorer(cohen_kappa_score, weights='quadratic')

print('------------------------------------------------------')

for dataset in all_dataset:
    # apply StandardScaler
    scaler = StandardScaler(copy=True, with_mean=True, with_std=True).fit(dataset)
    dataset = scaler.fit_transform(dataset)

    # split dataset
    StratifiedKFold(n_splits=5)
    X_train, X_test, y_train, y_test = train_test_split(dataset, y, random_state=1,  shuffle=True, stratify=y)#, test_size=0.2, train_size=0.8)

    # method
    randm_src = RandomizedSearchCV(estimator=model, param_distributions = parameters, cv = 5, n_iter = 10, n_jobs=5, scoring=qwck_scorer, verbose=2)
    randm_src.fit(X_train, y_train)

    # output
    print(f'results of data set {dataset}')
    print('------------------------------------------------------')
    print(" Results from Random Search: " )
    print("\n The best estimator across ALL searched params:\n", randm_src.best_estimator_)
    print("\n The best score across ALL searched params:\n", randm_src.best_score_)
    print("\n The best parameters across ALL searched params:\n", randm_src.best_params_)
    print('------------------------------------------------------------------------------------------------------------------------------------------------------------------')



------------------------------------------------------
Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ................C=0.9081632653061225, max_iter=2000; total time=19.1min
[CV] END ................C=0.9081632653061225, max_iter=2000; total time=20.1min
[CV] END ................C=0.9081632653061225, max_iter=2000; total time=20.2min
[CV] END ................C=0.9081632653061225, max_iter=2000; total time=21.0min
[CV] END ................C=0.9081632653061225, max_iter=2000; total time=21.1min
[CV] END ................C=0.6877551020408164, max_iter=2000; total time=17.1min
[CV] END ................C=0.6877551020408164, max_iter=2000; total time=18.3min
[CV] END ................C=0.6877551020408164, max_iter=2000; total time=18.5min
[CV] END ................C=0.6877551020408164, max_iter=2000; total time=18.4min
[CV] END ................C=0.6877551020408164, max_iter=2000; total time=18.6min
[CV] END ................C=0.5591836734693878, max_iter=2000; total time=1

In [31]:
randm_src

RandomizedSearchCV(cv=5, estimator=LogisticRegression(random_state=0), n_jobs=5,
                   param_distributions={'C': array([0.1       , 0.11836735, 0.13673469, 0.15510204, 0.17346939,
       0.19183673, 0.21020408, 0.22857143, 0.24693878, 0.26530612,
       0.28367347, 0.30204082, 0.32040816, 0.33877551, 0.35714286,
       0.3755102 , 0.39387755, 0.4122449 , 0.43061224, 0.44897959,
       0.46734694, 0.48571429, 0.50408...
       0.55918367, 0.57755102, 0.59591837, 0.61428571, 0.63265306,
       0.65102041, 0.66938776, 0.6877551 , 0.70612245, 0.7244898 ,
       0.74285714, 0.76122449, 0.77959184, 0.79795918, 0.81632653,
       0.83469388, 0.85306122, 0.87142857, 0.88979592, 0.90816327,
       0.92653061, 0.94489796, 0.96326531, 0.98163265, 1.        ]),
                                        'max_iter': [2000]},
                   scoring=make_scorer(cohen_kappa_score, weights=quadratic),
                   verbose=2)

## Score

In [32]:
#sk_cohen_kappa = cohen_kappa_score(y_prediction, y_test, weights='quadratic')
#print(f'quadratic cohen kappa: {sk_cohen_kappa}')

In [33]:
#from sklearn.metrics import confusion_matrix

In [34]:
#confusion_matrix(y_test, y_prediction)

In [35]:
#df_y[0].value_counts()

In [36]:
#len(y_test)